In [105]:
import pandas as pd
import requests
from sklearn.preprocessing import StandardScaler
import re
import difflib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.callbacks import EarlyStopping

In [74]:
# Lista de nombres de archivos de los conjuntos de datos locales
archivos = ["/content/dataset/movies.csv"]


In [75]:
# Lista para almacenar DataFrames individuales
dataframes = []
try:
    for archivo in archivos:
        # Leer cada conjunto de datos y almacenarlo en un DataFrame
        df = pd.read_csv(archivo, sep=',', encoding='utf-8')


    # Realizar operaciones con el DataFrame combinado
    print(df.head())  # Muestra las primeras filas del DataFrame combinado

except FileNotFoundError as e:
    print(f"Error: Archivo no encontrado: {str(e)}")

# Ahora tienes un DataFrame combinado con todos los datos



                                             name rating      genre  year  \
0                                     The Shining      R      Drama  1980   
1                                 The Blue Lagoon      R  Adventure  1980   
2  Star Wars: Episode V - The Empire Strikes Back     PG     Action  1980   
3                                       Airplane!     PG     Comedy  1980   
4                                      Caddyshack      R     Comedy  1980   

                        released  score      votes         director  \
0  June 13, 1980 (United States)    8.4   927000.0  Stanley Kubrick   
1   July 2, 1980 (United States)    5.8    65000.0   Randal Kleiser   
2  June 20, 1980 (United States)    8.7  1200000.0   Irvin Kershner   
3   July 2, 1980 (United States)    7.7   221000.0     Jim Abrahams   
4  July 25, 1980 (United States)    7.3   108000.0     Harold Ramis   

                    writer            star         country      budget  \
0             Stephen King  Jack Nic

In [76]:
longitud_con_len = len(df)
longitud_con_len


7668

In [77]:
# Encuentra las filas con valores NaN
filas_con_nan = df[df.isna().any(axis=1)]

# Imprime las primeras 5 filas con valores NaN
primeras_5_filas_nan = filas_con_nan.head(5)
print(primeras_5_filas_nan)

               name rating   genre  year                           released  \
16             Fame      R   Drama  1980       May 16, 1980 (United States)   
19       Stir Crazy      R  Comedy  1980  December 12, 1980 (United States)   
24     Urban Cowboy     PG   Drama  1980       June 6, 1980 (United States)   
25   Altered States      R  Horror  1980  December 25, 1980 (United States)   
26  Little Darlings      R  Comedy  1980     March 21, 1980 (United States)   

    score    votes        director              writer           star  \
16    6.6  21000.0     Alan Parker    Christopher Gore    Eddie Barth   
19    6.8  26000.0  Sidney Poitier  Bruce Jay Friedman    Gene Wilder   
24    6.4  14000.0   James Bridges        Aaron Latham  John Travolta   
25    6.9  33000.0     Ken Russell     Paddy Chayefsky   William Hurt   
26    6.5   5100.0     Ron Maxwell           Kimi Peck   Tatum O'Neal   

          country  budget        gross  \
16  United States     NaN   21202829.0   
19

In [78]:
df = df.dropna()


In [79]:
 len(df)

5421

In [80]:
    columnas_a_eliminar = ['released', 'gross', 'runtime']

    df = df.drop(columnas_a_eliminar, axis=1)


In [81]:
# Imprime las columnas con sus tipos de datos
df.dtypes

name         object
rating       object
genre        object
year          int64
score       float64
votes       float64
director     object
writer       object
star         object
country      object
budget      float64
company      object
dtype: object

In [82]:
df

,name,rating,genre,year,score,votes,director,writer,star,country,budget,company
0,The Shining,R,Drama,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,Warner Bros.
1,The Blue Lagoon,R,Adventure,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,Columbia Pictures
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,Lucasfilm
3,Airplane!,PG,Comedy,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,Paramount Pictures
4,Caddyshack,R,Comedy,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,Orion Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...
7648,Bad Boys for Life,R,Action,2020,6.6,140000.0,Adil El Arbi,Peter Craig,Will Smith,United States,90000000.0,Columbia Pictures
7649,Sonic the Hedgehog,PG,Action,2020,6.5,102000.0,Jeff Fowler,Pat Casey,Ben Schwartz,United States,85000000.0,Paramount Pictures
7650,Dolittle,PG,Adventure,2020,5.6,53000.0,Stephen Gaghan,Stephen Gaghan,Robert Downey Jr.,United States,175000000.0,Universal Pictures
7651,The Call of the Wild,PG,Adventure,2020,6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,135000000.0,20th Century Studios


In [97]:
columnas_a_normalizar = ['score']
scaler = StandardScaler()
df2 = df.copy()  # Crea una copia de df1 para df2
df2[columnas_a_normalizar] = scaler.fit_transform(df2[columnas_a_normalizar])
df2

,name,rating,genre,year,score,votes,director,writer,star,country,budget,company
0,The Shining,R,Drama,1980,2.085455,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,Warner Bros.
1,The Blue Lagoon,R,Adventure,1980,-0.615448,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,Columbia Pictures
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,2.397098,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,Lucasfilm
3,Airplane!,PG,Comedy,1980,1.358289,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,Paramount Pictures
4,Caddyshack,R,Comedy,1980,0.942765,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,Orion Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...
7648,Bad Boys for Life,R,Action,2020,0.215599,140000.0,Adil El Arbi,Peter Craig,Will Smith,United States,90000000.0,Columbia Pictures
7649,Sonic the Hedgehog,PG,Action,2020,0.111718,102000.0,Jeff Fowler,Pat Casey,Ben Schwartz,United States,85000000.0,Paramount Pictures
7650,Dolittle,PG,Adventure,2020,-0.823209,53000.0,Stephen Gaghan,Stephen Gaghan,Robert Downey Jr.,United States,175000000.0,Universal Pictures
7651,The Call of the Wild,PG,Adventure,2020,0.423361,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,135000000.0,20th Century Studios


In [110]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)


In [112]:
label_encoder_genre = LabelEncoder()
label_encoder_director = LabelEncoder()

In [114]:
# Aplicar codificación de etiquetas a 'genre' y 'director' en ambos conjuntos (entrenamiento y prueba)
X_train['genre'] = label_encoder_genre.fit_transform(X_train['genre'])
X_train['director'] = label_encoder_director.fit_transform(X_train['director'])

X_test['genre'] = label_encoder_genre.transform(X_test['genre'])
X_test['director'] = label_encoder_director.transform(X_test['director'])

ValueError: ignored

In [102]:
movie_id = Input(shape=[1])
genre_emb = Embedding(10, 5)(X_train['genre'])
director_emb = Embedding(1000, 5)(X_train['director'])

AttributeError: ignored